In [45]:
import pandas as pd
from libpysal.cg import KDTree, RADIUS_EARTH_MILES
from libpysal.weights import KNN
from spreg import ML_Lag

In [46]:
url = ('data/colleges_covid.csv')

# specify index cols we need only for identification -- not modeling
df = pd.read_csv(url, index_col=['unitid', 'instnm'])
 

# we'll `pop` renaming columns so they're no longer in our dataframe
x = df.copy().dropna(how='any')

# # tree object is the main input to nearest neighbors
tree = KDTree(
    data=zip(x.pop('longitude'), x.pop('latitude')),
    # default is euclidean, but we want to use arc or haversine distance
    distance_metric='arc',
    radius=RADIUS_EARTH_MILES
)
nn = KNN(tree, k=5)

y = x.pop('covid_cases_per_cap_jul312020')

# # spreg only accepts numpy arrays or lists as arguments
mod = ML_Lag(
    y=y.to_numpy(),
    x=x.to_numpy(),
    w=nn,
    name_y=y.name,
    name_x=x.columns.tolist()
)

# # results
print(mod.summary)

/home/asus/PycharmProjects/DS/GSA/taska/lib/python3.10/site-packages/libpysal/weights/weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 6 disconnected components.
  warnings.warn(message)
/home/asus/PycharmProjects/DS/GSA/taska/lib/python3.10/site-packages/spreg/ml_lag.py:634: RuntimeWarning: invalid value encountered in log
  jacob = np.log(np.linalg.det(a))


REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :covid_cases_per_cap_jul312020                Number of Observations:        2797
Mean dependent var  :      0.0129                Number of Variables   :           4
S.D. dependent var  :      0.0089                Degrees of Freedom    :        2793
Pseudo R-squared    :      0.7256
Spatial Pseudo R-squared:  0.0177
Log likelihood      :  10729.6452
Sigma-square ML     :      0.0000                Akaike info criterion :  -21451.290
S.E of regression   :      0.0048                Schwarz criterion     :  -21427.545

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
----------------------------------------------